In [ ]:
# Install the required packages
!pip install pyrealsense2

In [ ]:
import cv2
import numpy as np
import pyrealsense2 as rs
import torch
from torchvision import transforms
from PIL import Image
import torch.nn.functional as F
from torchvision.models import resnet18
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,Input,Dropout,GlobalAveragePooling2D,Flatten,Conv2D,BatchNormalization,Activation,MaxPooling2D
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
import torch.nn as nn

## 1) Checking the working of intel real sense camera module

In [ ]:
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)

try:
    while True:
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        
        if not depth_frame or not color_frame:
            continue
        
        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        
        # Apply colormap on depth image (optional)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
        
        # Stack both images horizontally
        images = np.hstack((color_image, depth_colormap))
        
        # Show images
        cv2.imshow('RealSense Camera', images)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # Stop streaming
    pipeline.stop()
    cv2.destroyAllWindows()


## 2) Creating Image model for video emotion

In [6]:


class Face_Emotion_CNN(nn.Module):
  def __init__(self):
    super(Face_Emotion_CNN, self).__init__()
    self.cnn1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3)
    self.cnn2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3)
    self.cnn3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3)
    self.cnn4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
    self.cnn5 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
    self.cnn6 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3)
    self.cnn7 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3)
    self.relu = nn.ReLU()
    self.pool1 = nn.MaxPool2d(2, 1)
    self.pool2 = nn.MaxPool2d(2, 2)
    self.cnn1_bn = nn.BatchNorm2d(8)
    self.cnn2_bn = nn.BatchNorm2d(16)
    self.cnn3_bn = nn.BatchNorm2d(32)
    self.cnn4_bn = nn.BatchNorm2d(64)
    self.cnn5_bn = nn.BatchNorm2d(128)
    self.cnn6_bn = nn.BatchNorm2d(256)
    self.cnn7_bn = nn.BatchNorm2d(256)
    self.fc1 = nn.Linear(1024, 512)
    self.fc2 = nn.Linear(512, 256)
    self.fc3 = nn.Linear(256, 7)
    self.dropout = nn.Dropout(0.3)
    self.log_softmax = nn.LogSoftmax(dim=1)
    
  def forward(self, x):
    x = self.relu(self.pool1(self.cnn1_bn(self.cnn1(x))))
    x = self.relu(self.pool1(self.cnn2_bn(self.dropout(self.cnn2(x)))))
    x = self.relu(self.pool1(self.cnn3_bn(self.cnn3(x))))
    x = self.relu(self.pool1(self.cnn4_bn(self.dropout(self.cnn4(x)))))
    x = self.relu(self.pool2(self.cnn5_bn(self.cnn5(x))))
    x = self.relu(self.pool2(self.cnn6_bn(self.dropout(self.cnn6(x)))))
    x = self.relu(self.pool2(self.cnn7_bn(self.dropout(self.cnn7(x)))))
    
    x = x.view(x.size(0), -1)
    
    x = self.relu(self.dropout(self.fc1(x)))
    x = self.relu(self.dropout(self.fc2(x)))
    x = self.log_softmax(self.fc3(x))
    return x

  def count_parameters(self):
    return sum(p.numel() for p in self.parameters() if p.requires_grad)


if __name__ == '__main__':
    bn_model = Face_Emotion_CNN()
    x = torch.randn(1,1,48,48)
    print('Shape of output = ',bn_model(x).shape)
    print('No of Parameters of the BatchNorm-CNN Model =',bn_model.count_parameters())

Shape of output =  torch.Size([1, 7])
No of Parameters of the BatchNorm-CNN Model = 1642935


In [7]:
import cv2
import torch
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import argparse
import os
# from model import *


def load_trained_model(model_path):
    model = Face_Emotion_CNN()
    model.load_state_dict(torch.load(model_path, map_location=lambda storage, loc: storage), strict=False)
    return model

def FER_live_cam():

    model = load_trained_model('FER_trained_model.pt')
    
    emotion_dict = {0: 'neutral', 1: 'happiness'}


    val_transform = transforms.Compose([
        transforms.ToTensor()])

    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        faces = face_cascade.detectMultiScale(frame)
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,0), 2)
            resize_frame = cv2.resize(gray[y:y + h, x:x + w], (48, 48))
            X = resize_frame/256
            X = Image.fromarray((X))
            X = val_transform(X).unsqueeze(0)
            with torch.no_grad():
                model.eval()
                log_ps = model.cpu()(X)
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                pred = emotion_dict[int(top_class.numpy())]
            cv2.putText(frame, pred, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1)
        
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()



if __name__ == "__main__":

    FER_live_cam()

## Quantizing the model

In [9]:
import cv2
import torch
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import argparse
import os
from torch.quantization import quantize_dynamic

def load_trained_model(model_path):
    model = Face_Emotion_CNN()
    model.load_state_dict(torch.load(model_path, map_location=lambda storage, loc: storage), strict=False)
    return model

def FER_live_cam():

    model = load_trained_model('FER_trained_model.pt')
    
    emotion_dict = {0: 'neutral', 1: 'happiness'}

    val_transform = transforms.Compose([
        transforms.ToTensor()])

    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        faces = face_cascade.detectMultiScale(frame)
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,0), 2)
            resize_frame = cv2.resize(gray[y:y + h, x:x + w], (48, 48))
            X = resize_frame/256
            X = Image.fromarray((X))
            X = val_transform(X).unsqueeze(0)
            with torch.no_grad():
                model.eval()
                log_ps = model.cpu()(X)
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                pred = emotion_dict[int(top_class.numpy())]
            cv2.putText(frame, pred, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1)
        
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # Save the original model
    torch.save(model.state_dict(), 'original_model.pth')

    # Quantize the model dynamically
    quantized_model = quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)

    # Save the quantized model
    torch.save(quantized_model.state_dict(), 'quantized_model.pth')

    # Print the file sizes
    original_size = os.path.getsize('original_model.pth')/(1024*1024)
    quantized_size = os.path.getsize('quantized_model.pth')/(1024*1024)
    print("Original model size:", round(original_size, 2), "MB")
    print("Quantized model size:", round(quantized_size, 2), "MB")
if __name__ == "__main__":
    FER_live_cam()


Original model size: 6.29 MB
Quantized model size: 4.41 MB


# Transfer Learning on custom dataset

In [2]:
import torch.nn as nn

class Face_Emotion_CNN(nn.Module):
    def __init__(self):
        super(Face_Emotion_CNN, self).__init__()
        self.cnn1 = nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1)  # Update input channels to 3
        self.cnn1_bn = nn.BatchNorm2d(8)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.cnn2 = nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1)
        self.cnn2_bn = nn.BatchNorm2d(16)
        self.dropout = nn.Dropout(p=0.5)
        self.cnn3 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.cnn3_bn = nn.BatchNorm2d(32)
        self.fc1 = nn.Linear(32 * 6 * 6, 128)
        self.fc1_bn = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 64)
        self.fc2_bn = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 7)  # Output layer with 7 units for 7 classes
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.pool1(self.cnn1_bn(self.cnn1(x))))
        x = self.relu(self.pool1(self.cnn2_bn(self.dropout(self.cnn2(x)))))
        x = self.relu(self.pool1(self.cnn3_bn(self.cnn3(x))))
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1_bn(self.fc1(x)))
        x = self.relu(self.fc2_bn(self.fc2(x)))
        x = self.fc3(x)
        return x


In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
# from model import Face_Emotion_CNN

# Define data transformations
train_transform = transforms.Compose([
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    # Add more data augmentation transformations if needed
])

batch_size = 64
learning_rate = 0.001
num_epochs = 20

# Define dataset and dataloader
train_dataset = ImageFolder(root=r"Path_to_dataset", transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Define model
model = Face_Emotion_CNN()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Save the trained model
torch.save(model.state_dict(), 'FER_trained_model_new.pt')

print("Training complete. Model saved.")


Epoch [1/20], Loss: 0.7613
Epoch [2/20], Loss: 0.7313
Epoch [3/20], Loss: 0.7106
Epoch [4/20], Loss: 0.6982
Epoch [5/20], Loss: 0.7145
Epoch [6/20], Loss: 0.6845
Epoch [7/20], Loss: 0.6901
Epoch [8/20], Loss: 0.6936
Epoch [9/20], Loss: 0.6952
Epoch [10/20], Loss: 0.7116
Epoch [11/20], Loss: 0.7086
Epoch [12/20], Loss: 0.7120
Epoch [13/20], Loss: 0.7093
Epoch [14/20], Loss: 0.7016
Epoch [15/20], Loss: 0.7066
Epoch [16/20], Loss: 0.7073
Epoch [17/20], Loss: 0.7003
Epoch [18/20], Loss: 0.6880
Epoch [19/20], Loss: 0.7081
Epoch [20/20], Loss: 0.6985
Training complete. Model saved.


In [3]:
import cv2
import torch
import torchvision.transforms as transforms
# from model import Face_Emotion_CNN  # Import your model here

# Load the trained model
model = Face_Emotion_CNN()
model.load_state_dict(torch.load('FER_trained_model_new.pt'))
model.eval()

# Define the transformation for input images
val_transform = transforms.Compose([
    transforms.ToTensor()
])

# Define the emotion labels
emotion_dict = {0: 'happy', 1: 'neutral'}

# Open the camera
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    gray = frame
    # Detect faces in the frame
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    # Process each detected face
    for (x, y, w, h) in faces:
        # Extract the face region from the frame
        face_roi = gray[y:y + h, x:x + w]

        # Resize the face region to match the input size of the model
        resized_face = cv2.resize(face_roi, (48, 48))

        # Normalize and transform the face image
        normalized_face = resized_face / 255.0  # Normalize to [0,1]
        transformed_face = val_transform(normalized_face)

        # Convert the input tensor to the same data type as the bias parameter
        input_tensor = transformed_face.float()

        # Add batch dimension and convert to tensor
        input_tensor = torch.unsqueeze(input_tensor, dim=0)

        # Perform inference
        with torch.no_grad():
            model.eval()
            outputs = model(input_tensor)

            # Get the predicted emotion label
            _, predicted = torch.max(outputs.data, 1)
            emotion_label = emotion_dict[predicted.item()]

        # Draw the bounding box and emotion label on the frame
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.putText(frame, emotion_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Emotion Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()


# Inference

In [20]:
import cv2
import torch
import torch.nn as nn
from torchvision import models, transforms
import numpy as np

# Define the model architecture
class FERPlus(nn.Module):
    def __init__(self, num_classes=8):
        super(FERPlus, self).__init__()
        self.resnet50 = models.resnet50(pretrained=True)
        self.resnet50.fc = nn.Linear(self.resnet50.fc.in_features, num_classes)
    
    def forward(self, x):
        return self.resnet50(x)

# Initialize the model
model = FERPlus(num_classes=2)  # 8 classes for FERPlus dataset


In [21]:
model.eval()

FERPlus(
  (resnet50): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
         

In [23]:
import cv2
import numpy as np
import torch
from torchvision import transforms
from PIL import Image
import torch.nn.functional as F
from torchvision.models import resnet50

# Load the pre-trained ResNet50 model for emotion detection
model = resnet50(pretrained=True)
model.fc = torch.nn.Linear(2048, 7)  # 7 classes for emotions
model.eval()

# Load the RealSense camera
import pyrealsense2 as rs

# Initialize RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
pipeline.start(config)

# Define emotion labels
# emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
emotion_labels = ['Happy', 'Neutral']

# Function to preprocess image for ResNet50
def preprocess_image(image):
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = Image.fromarray(image)
    image = preprocess(image).unsqueeze(0)
    return image

try:
    while True:
        # Wait for a color frame
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        
        if not color_frame:
            continue
        
        # Convert color image to numpy array
        color_image = np.asanyarray(color_frame.get_data())

        # Convert to grayscale for face detection
        gray = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)

        # Use Haar cascade to detect faces
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Iterate through faces
        for (x, y, w, h) in faces:
            # Extract face region
            face = color_image[y:y+h, x:x+w]
            
            # Preprocess image for ResNet50
            face_tensor = preprocess_image(face)
            
            # Perform emotion detection
            with torch.no_grad():
                outputs = model(face_tensor)
                _, predicted = torch.max(outputs.data, 1)
                emotion = emotion_labels[predicted.item()]
            
            # Draw rectangle around the face
            cv2.rectangle(color_image, (x, y), (x+w, y+h), (255, 0, 0), 2)
            
            # Put emotion label on the frame
            cv2.putText(color_image, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        
        # Show the frame with emotions
        cv2.imshow('Emotion Detection', color_image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
q
            
finally:
    # Stop streaming
    pipeline.stop()
    cv2.destroyAllWindows()


RuntimeError: No device connected

## Creating Model for audio to emotion

In [1]:
import sys
import torch
import speech_recognition as sr
from transformers import BertTokenizer, BertForSequenceClassification
import pygame

# Initialize pygame
pygame.init()

# Load pre-trained model and tokenizer
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# Define the recognizer
recognizer = sr.Recognizer()

# Function to start recording
def start_recording():
    # Record audio from the microphone for 10 seconds
    print("Recording...")
    with sr.Microphone() as source:
        recognizer.adjust_for_ambient_noise(source)  # Adjust for ambient noise
        audio_data = recognizer.record(source, duration=2)

    # Recognize speech using Google Speech Recognition
    try:
        text = recognizer.recognize_google(audio_data)
        print("Text:", text)
        #emo = predict_emotion(text)
        predicted_label = predict_emotion(text)
        print(predicted_label)
        recorded_text = f"Recorded Text: {text}"
        predicted_text = f"Predicted Emotion: {predicted_label}"
        print(predicted_text)
        draw_text(recorded_text, predicted_label)
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand the audio.")
        draw_text("Google Speech Recognition could not understand the audio.", "")
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))
        draw_text("Could not request results from Google Speech Recognition service.", "")

# Function to predict emotion from text
def predict_emotion(text):
    # Tokenize input text
    tokens = tokenizer(text, return_tensors="pt")

    # Perform inference
    with torch.no_grad():
        outputs = model(**tokens)

    # Get predicted label
    predicted_label = torch.argmax(outputs.logits, dim=1).item()

    # Map predicted label to emotion
    emotion_mapping = {
        0: "SAD",
        1: "SAD",
        2: "Neutral",
        3: "HAPPY",
        4: "Excited"
    }

    # Return predicted emotion
    return emotion_mapping.get(predicted_label, "Unknown")

# Function to draw text on the screen
def draw_text(recorded_text, predicted_text):
    screen.fill((255, 255, 255))
    font = pygame.font.Font(None, 30)
    text_surface1 = font.render(recorded_text, True, (0, 0, 0))
    text_surface2 = font.render(predicted_text, True, (0, 0, 0))
    screen.blit(text_surface1, (50, 50))
    screen.blit(text_surface2, (50, 100))
    pygame.display.flip()

# Set up the screen
WIDTH, HEIGHT = 600, 200
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Emotion Recognition")

# Set up the button
button_font = pygame.font.Font(None, 36)
button_text = button_font.render("Start Recording", True, (0, 0, 0))
button_rect = button_text.get_rect(center=(WIDTH // 2, HEIGHT // 2))

# Main loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            if button_rect.collidepoint(event.pos):
                start_recording()

    # Clear the screen
    screen.fill((255, 255, 255))

    # Draw the button
    pygame.draw.rect(screen, (200, 200, 200), button_rect)
    pygame.draw.rect(screen, (0, 0, 0), button_rect, 2)
    screen.blit(button_text, button_rect.topleft)

    # Update the display
    pygame.display.flip()

# Quit pygame
pygame.quit()
sys.exit()

pygame 2.5.2 (SDL 2.28.3, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html
Recording...
Text: it was a good day today
HAPPY
Predicted Emotion: HAPPY


SystemExit: 

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [7]:
# Get the size of the model
model_size_bytes = sum(p.numel() for p in model.parameters() if p.requires_grad)
model_size_mb = model_size_bytes / (1024 * 1024)
print("Model size:", model_size_mb, "MB")

Model size: 159.60718250274658 MB


In [13]:
import torch.quantization

# Quantize the model 
quantized_model = torch.quantization.quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)


In [14]:
import torch

# Get the size of the quantized model
quantized_model_size_bytes = sum(p.numel() for p in quantized_model.parameters() if p.requires_grad)
quantized_model_size_mb = quantized_model_size_bytes / (1024 * 1024)
print("Quantized model size:", quantized_model_size_mb, "MB")


Quantized model size: 77.961181640625 MB
